# Obtener base de datos de Masa salarial por sector económico

En esta libreta se trabajan los datos previamente filtrados para la Ciudad de México (para más información, ver libreta `Filtrar_CDMX.ipynb`). De ellos se obtiene un base de datos que contiene la masa salarial por tipo de puesto de trabajo:

`{'masa_sal_ta':  'Masa salarial asociada a puestos de trabajo afiliados',
'masa_sal_teu': 'Masa salarial asociada a puestos de trabajo eventuales urbanos',
'masa_sal_tec': 'Masa salarial asociada a puestos de trabajo eventuales del campo',
'masa_sal_tpu': 'Masa salarial asociada a puestos de trabajo permanentes urbanos',
'masa_sal_tpc': 'Masa salarial asociada a puestos de trabajo permanentes del campo'}`

y por sector económico `sector_economico_2_2pos` del diccionario de datos del IMSS (las definiciones de cada sector se encuentran más abajo en esta misma libreta).

Cabe mencionar que la etiqueta `masa_sal_ta` contiene la suma de las demás etiqutas de tipo de masa salarial, es decir, `masa_sal_ta` contiene el total de la masa salarial de cada sector económico.

In [9]:
# Importar bibliotecas y agregar rutas al sys.path

import os
import sys
import inspect
import numpy as np
import pandas as pd

parentdir = os.path.abspath('..')
grandparentdir=os.path.dirname(parentdir)
ggpdir=os.path.dirname(grandparentdir)

sys.path.insert(0, parentdir)
sys.path.insert(0, grandparentdir)
sys.path.insert(0, ggpdir)

In [10]:
#Obtener nombres de archivos del IMSS para CDMX (previamente filtrados en la libreta Filtrar_CDMX)
mypath=parentdir+'/IMSS_CDMX/'
asg_files= sorted([f for f in os.listdir(mypath) if f.startswith('asg')])
asg_files

['asg-2010-01_CDMX.csv',
 'asg-2010-02_CDMX.csv',
 'asg-2010-03_CDMX.csv',
 'asg-2010-04_CDMX.csv',
 'asg-2010-05_CDMX.csv',
 'asg-2010-06_CDMX.csv',
 'asg-2010-07_CDMX.csv',
 'asg-2010-08_CDMX.csv',
 'asg-2010-09_CDMX.csv',
 'asg-2010-10_CDMX.csv',
 'asg-2010-11_CDMX.csv',
 'asg-2010-12_CDMX.csv',
 'asg-2011-01_CDMX.csv',
 'asg-2011-02_CDMX.csv',
 'asg-2011-03_CDMX.csv',
 'asg-2011-04_CDMX.csv',
 'asg-2011-05_CDMX.csv',
 'asg-2011-06_CDMX.csv',
 'asg-2011-07_CDMX.csv',
 'asg-2011-08_CDMX.csv',
 'asg-2011-09_CDMX.csv',
 'asg-2011-10_CDMX.csv',
 'asg-2011-11_CDMX.csv',
 'asg-2011-12_CDMX.csv',
 'asg-2012-01_CDMX.csv',
 'asg-2012-02_CDMX.csv',
 'asg-2012-03_CDMX.csv',
 'asg-2012-04_CDMX.csv',
 'asg-2012-05_CDMX.csv',
 'asg-2012-06_CDMX.csv',
 'asg-2012-07_CDMX.csv',
 'asg-2012-08_CDMX.csv',
 'asg-2012-09_CDMX.csv',
 'asg-2012-10_CDMX.csv',
 'asg-2012-11_CDMX.csv',
 'asg-2012-12_CDMX.csv',
 'asg-2013-01_CDMX.csv',
 'asg-2013-02_CDMX.csv',
 'asg-2013-03_CDMX.csv',
 'asg-2013-04_CDMX.csv',


# Obtener las etiquetas y definiciones del sector económico 2, desde el diccionario de datos

En la siguiente celda, se muestra como se ven los datos filtrados para CDMX en un mes y año específico

In [11]:
# Leer datos previamente filtrados para CDMX

df_asg_2023_01_CDMX=pd.read_csv(mypath+'asg-2023-01_CDMX.csv',
                                low_memory=False)
df_asg_2023_01_CDMX

,Unnamed: 0,cve_delegacion,cve_subdelegacion,cve_entidad,cve_municipio,sector_economico_1,sector_economico_2,sector_economico_4,tamaño_patron,sexo,...,ta_sal,teu_sal,tec_sal,tpu_sal,tpc_sal,masa_sal_ta,masa_sal_teu,masa_sal_tec,masa_sal_tpu,masa_sal_tpc
0,0,39,11,9,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0.00,0.0,0.0,0.00,0.0
1,1,39,11,9,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0.00,0.0,0.0,0.00,0.0
2,2,39,11,9,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0.00,0.0,0.0,0.00,0.0
3,3,39,11,9,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0.00,0.0,0.0,0.00,0.0
4,4,39,11,9,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0.00,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499406,499406,40,58,9,NaN,9.0,99.0,9900.0,S3,2,...,1,0,0,1,0,1980.40,0.0,0.0,1980.40,0.0
499407,499407,40,58,9,NaN,9.0,99.0,9900.0,S3,2,...,1,0,0,1,0,2230.94,0.0,0.0,2230.94,0.0
499408,499408,40,58,9,NaN,9.0,99.0,9900.0,S3,2,...,1,0,0,1,0,2405.50,0.0,0.0,2405.50,0.0
499409,499409,40,58,9,NaN,9.0,99.0,9900.0,S3,2,...,2,0,0,2,0,4811.00,0.0,0.0,4811.00,0.0


En la siguiente celda, se muestra la hoja del diccionario de datos correspondiente al sector económico 2. Luego se muestran las etiquetas numéricas que representan a cada sector, y más adelante se imprime la definición de cada sector.

In [12]:
# Sector economico 2 de diccionario de datos

df_sec2=pd.read_excel('../row_data/diccionario_de_datos_1.xlsx',sheet_name='sector 2',skiprows=[0])
# df_rango_sal['rango_salarial'].fillna('NA',inplace=True)
df_sec2

,sector_economico_1,sector_economico_2,sector_economico_2_2pos,descripción sector_economico_2
0,0,1,1,Agricultura
1,0,2,2,Ganadería
2,0,3,3,Silvicultura
3,0,4,4,Pesca
4,0,5,5,Caza
...,...,...,...,...
57,9,1,91,"Servicios de enseñanza, investigación científi..."
58,9,2,92,"Servicios médicos, asistencia social y veterin..."
59,9,3,93,"Agrupaciones mercantiles, profesionales, cívic..."
60,9,4,94,Servicios de administración pública y segurida...


In [13]:
sec2=df_sec2['sector_economico_2_2pos'].values
sec2

array([ 1,  2,  3,  4,  5, 11, 12, 13, 14, 20, 21, 22, 23, 24, 25, 26, 27,
       28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 50, 51, 61,
       62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 81, 82, 83,
       84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 99])

In [14]:
for indicador in df_sec2['sector_economico_2_2pos'].values:
    print(str(indicador)+':= ',df_sec2[df_sec2['sector_economico_2_2pos']==indicador]['descripción sector_economico_2'].values[0])

1:=  Agricultura
2:=  Ganadería
3:=  Silvicultura
4:=  Pesca
5:=  Caza
11:=  Extracción  y  beneficio de carbón mineral, grafito y minerales no metálicos; excepto sal
12:=  Exploración y extracción  de  petróleo crudo y gas natural (no incluye a Pemex)                                                                                     
13:=  Extracción y beneficio de minerales metálicos    
14:=  Explotación de sal                   
20:=  Fabricación de alimentos
21:=  Elaboración de bebidas
22:=  Beneficio y fabricación de productos de tabaco
23:=  Industria textil
24:=  Confección de prendas de vestir y otros artículos confeccionados con textiles y otros materiales; excepto calzado
25:=  Fabricación de  calzado e industria del cuero
26:=  Industria y productos de madera y corcho; excepto muebles
27:=  Fabricación y reparación de muebles de madera y sus partes; excepto los de metal y plástico moldeado
28:=  Industria del papel                                                          

In [15]:
masa_sal_columns = [col for col in df_asg_2023_01_CDMX.columns if col.startswith('masa')]

masa_sal_columns

['masa_sal_ta', 'masa_sal_teu', 'masa_sal_tec', 'masa_sal_tpu', 'masa_sal_tpc']

# Seleccionar los datos donde el sector economico corresponde a un indicador específico y a los indicadores de masa salarial

En la siguiente celda se obtiene la masa salarial para los distintos tipos de puestos de trabajo, para una etiqueta del sector económico 2 en específico. Este proceso se va a iterar en cada base de datos para crear el dataframe final.

In [16]:
# Seleccionar los datos donde el sector economico corresponde a un indicador específico
# y a los indicadores de masa salarial

# masa_sal_ta:=  Masa salarial asociada a puestos de trabajo afiliados
# masa_sal_teu:=  Masa salarial asociada a puestos de trabajo eventuales urbanos
# masa_sal_tec:=  Masa salarial asociada a puestos de trabajo eventuales del campo
# masa_sal_tpu:=  Masa salarial asociada a puestos de trabajo permanentes urbanos
# masa_sal_tpc:=  Masa salarial asociada a puestos de trabajo permanentes del campo


df_masa_sal=df_asg_2023_01_CDMX[df_asg_2023_01_CDMX['sector_economico_2']==11][masa_sal_columns+['sector_economico_2']]
df_masa_sal


,masa_sal_ta,masa_sal_teu,masa_sal_tec,masa_sal_tpu,masa_sal_tpc,sector_economico_2
59,492.90,0.00,0.0,492.90,0.0,11.0
60,367.26,367.26,0.0,0.00,0.0,11.0
61,367.26,367.26,0.0,0.00,0.0,11.0
62,1101.78,1101.78,0.0,0.00,0.0,11.0
63,699.54,699.54,0.0,0.00,0.0,11.0
...,...,...,...,...,...,...
429739,301.77,0.00,0.0,301.77,0.0,11.0
429740,543.19,0.00,0.0,543.19,0.0,11.0
429741,435.34,0.00,0.0,435.34,0.0,11.0
429742,217.67,0.00,0.0,217.67,0.0,11.0


# Loop principal

A continuación, se itera el proceso anterior para cada tipo de puesto trabajo y cada etiqueta del sector económico 2. Estos datos se guardan un dataframe en el que a cada Fecha (`aaaa-mm`) le corresponde la masa salarial por tipo de puesto de trabajo (e.g., `masa_sal_ta`) y la masa salarial por tipo de puesto de trabajo por sector económico 2 (e.g., `masa_sal_ta_1`). En cada paso del loop, se especifica la base de datos que se está trabajado y se imprime el tiempo de procesamiento. Al final, en la siguiente celda, se escribe el dataframe con nombre `'Masa_sal_Sec_2.csv'` en la carpeta creada con nombre `'IMSS_CDMX_Masa_sal_sec2'`.

In [17]:
fecha_masa_sal_sec_2=['Fecha']

for ms in masa_sal_columns:
    fecha_masa_sal_sec_2.append(ms)

for ms in masa_sal_columns:
    for s2 in sec2:
        ms_s2=ms+'_'+str(s2)
        fecha_masa_sal_sec_2.append(ms_s2)

df_fecha_ms_s2=pd.DataFrame(columns=fecha_masa_sal_sec_2)
df_fecha_ms_s2['Fecha']=[f[4:11] for f in asg_files]
df_fecha_ms_s2

,Fecha,masa_sal_ta,masa_sal_teu,masa_sal_tec,masa_sal_tpu,masa_sal_tpc,masa_sal_ta_1,masa_sal_ta_2,masa_sal_ta_3,masa_sal_ta_4,...,masa_sal_tpc_85,masa_sal_tpc_86,masa_sal_tpc_87,masa_sal_tpc_88,masa_sal_tpc_89,masa_sal_tpc_91,masa_sal_tpc_92,masa_sal_tpc_93,masa_sal_tpc_94,masa_sal_tpc_99
0,2010-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,2023-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,2023-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,2023-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
import time

t00 = time.time()

write_dir=parentdir+'/IMSS_CDMX_Masa_sal_sec2/'

if not os.path.exists(write_dir):
    os.makedirs(write_dir)

for asg_data in asg_files:
    t01 = time.time()
    
    print('Procesando:',asg_data)
#     Leer dataframe
    df_asg=pd.read_csv(mypath+asg_data,low_memory=False)
    

    
    fecha=asg_data[4:11]
    
    for s2 in sec2:
        df_s2=df_asg[df_asg['sector_economico_2']==s2]
        for ms in masa_sal_columns:
            ms_s2=ms+'_'+str(s2)
            ms_s2_total=sum(df_s2[ms].values)
#             pt_total=sum(df_asg[df_asg['rango_salarial']=='W2']['ta'])
            
            df_fecha_ms_s2.loc[df_fecha_ms_s2['Fecha']==fecha,ms_s2]=ms_s2_total
        
    for ms in masa_sal_columns:
        ms_total=sum(df_asg[ms].values)
        
        df_fecha_ms_s2.loc[df_fecha_ms_s2['Fecha']==fecha,ms]=ms_total

    
    t11 = time.time()

    total11 = t11-t01 
    
    print(total11)

t10 = time.time()


total00 = t10-t00


print('Tiempo total=',total00)

df_fecha_ms_s2

Procesando: asg-2010-01_CDMX.csv
2.143366813659668
Procesando: asg-2010-02_CDMX.csv
2.5852739810943604
Procesando: asg-2010-03_CDMX.csv
1.4126880168914795
Procesando: asg-2010-04_CDMX.csv
1.3902342319488525
Procesando: asg-2010-05_CDMX.csv
1.4968292713165283
Procesando: asg-2010-06_CDMX.csv
1.609314203262329
Procesando: asg-2010-07_CDMX.csv
1.1949150562286377
Procesando: asg-2010-08_CDMX.csv
4.5709617137908936
Procesando: asg-2010-09_CDMX.csv
1.3881890773773193
Procesando: asg-2010-10_CDMX.csv
1.2447750568389893
Procesando: asg-2010-11_CDMX.csv
1.279900312423706
Procesando: asg-2010-12_CDMX.csv
1.3866608142852783
Procesando: asg-2011-01_CDMX.csv
1.3133800029754639
Procesando: asg-2011-02_CDMX.csv
1.2178089618682861
Procesando: asg-2011-03_CDMX.csv
1.2992260456085205
Procesando: asg-2011-04_CDMX.csv
1.2443697452545166
Procesando: asg-2011-05_CDMX.csv
1.2214269638061523
Procesando: asg-2011-06_CDMX.csv
1.172239065170288
Procesando: asg-2011-07_CDMX.csv
1.3398938179016113
Procesando: asg-

4.290406227111816
Tiempo total= 316.0705637931824


,Fecha,masa_sal_ta,masa_sal_teu,masa_sal_tec,masa_sal_tpu,masa_sal_tpc,masa_sal_ta_1,masa_sal_ta_2,masa_sal_ta_3,masa_sal_ta_4,...,masa_sal_tpc_85,masa_sal_tpc_86,masa_sal_tpc_87,masa_sal_tpc_88,masa_sal_tpc_89,masa_sal_tpc_91,masa_sal_tpc_92,masa_sal_tpc_93,masa_sal_tpc_94,masa_sal_tpc_99
0,2010-01,795708666.799878,62274037.740002,0.0,733434629.05993,0.0,165869.83,218018.56,5677.67,2695.38,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-02,799290269.619883,63216220.600001,0.0,736074049.019928,0.0,171166.49,254836.64,5547.32,3000.62,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-03,794064878.509875,65200137.260001,0.0,728864741.249944,0.0,169638.7,258697.97,5886.37,3195.62,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-04,797416477.279865,65627314.580002,0.0,731789162.699917,0.0,172838.26,272625.1,5886.37,3465.56,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2010-05,810774358.689839,67215692.950002,0.0,743558665.739866,0.0,173027.61,295062.58,5760.35,3530.56,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2022-12,2091462225.009816,200112151.930029,200.0,1891344446.789781,5426.29,967811.51,2152676.25,9825.14,41799.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156,2023-01,2212190132.400419,211398639.479976,217.67,2000785034.960322,6240.29,913801.14,1971717.96,8382.51,54368.88,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157,2023-02,2253413930.04037,217769245.859977,217.67,2035638226.220287,6240.29,906440.72,1962962.66,9027.57,54482.39,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
158,2023-03,2276352018.290335,224724555.809977,217.67,2051619758.240247,7486.57,874848.66,2013708.49,8168.78,42658.24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df_fecha_ms_s2.to_csv('../IMSS_CDMX_Masa_Salarial/Masa_sal_Sec_2.csv')